In [34]:


%load_ext autoreload
%autoreload 2

from pathlib import Path
import cv2
import torch
from dnafiber.inference import preprocess_image


from tqdm.auto import tqdm
import pandas as pd

from dnafiber.ui.utils import get_model, MODELS_ZOO
from dnafiber.data.intergrader import REVERSE_BLIND_MAPPING
from dnafiber.data.intergrader.analysis import (
    extract_properties_from_datadict,
    skeletonize_data_dict,
)
from skimage.segmentation import expand_labels
from dnafiber.data.intergrader.io import read_mask_from_path_gens
from dnafiber.data.intergrader.plot import imshow_compare


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
root = Path("C:/Users/cleme/Documents/Projects/DNAFiber/to_label_v1/Clement/common")
all_files = list(root.rglob("*/*.jpeg"))
all_files = sorted(all_files, key=lambda x: x.stem)[:-1]

modelname = list(MODELS_ZOO.keys())[1]
revision = MODELS_ZOO[modelname]
model = get_model("cuda", revision).cuda()
model.eval()

output = dict()
for i, file in enumerate(tqdm(all_files)):
    img_type  = REVERSE_BLIND_MAPPING[file.parent.stem]
    img = cv2.imread(str(file))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) / 255.0
    
    tensor = preprocess_image(img).float().cuda()
    with torch.no_grad():
        pred = model(tensor)
    pred = pred.argmax(1).squeeze().cpu().numpy()
    pred = expand_labels(pred, 1)
    # fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    # axs[0].imshow(img)
    # axs[0].set_title(f"{file.parent.stem} {file.stem}")
    # axs[1].imshow(pred, cmap=ListedColormap(['black', 'red', 'green']), interpolation="nearest")
    # plt.show()
    if img_type not in output:
        output[img_type] = []
    output[img_type].append(pred)


2025-04-24 14:24:59.769 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-24 14:24:59.771 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-24 14:24:59.771 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-24 14:25:00.288 Thread 'Thread-19': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-24 14:25:00.291 Thread 'Thread-19': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-24 14:25:02.015 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-24 14:25:02.016 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


  0%|          | 0/19 [00:00<?, ?it/s]

In [36]:
root_clement = "C:/Users/cleme/OneDrive/PostDoc/data/labelled/DNAFiber_Clement/export/colormaps/local/common"
root_yosra = "C:/Users/cleme/OneDrive/PostDoc/data/labelled/Yosra/export/colormaps/local/common"
root_hugo = "C:/Users/cleme/OneDrive/PostDoc/data/labelled/Hugo/export/colormaps/local/common"
root_santiago = "C:/Users/cleme/OneDrive/PostDoc/data/labelled/Santiago/export/colormaps/local/common"

yosra_files = list(Path(root_yosra).rglob("*/*.png"))
clement_files = list(Path(root_clement).rglob("*/*.png"))
hugo_files = list(Path(root_hugo).rglob("*/*.png"))
santiago_files = list(Path(root_santiago).rglob("*/*.png"))

yosra_files = sorted(yosra_files, key=lambda x: x.stem)[:-1]
clement_files = sorted(clement_files, key=lambda x: x.stem)[:-1]
hugo_files = sorted(hugo_files, key=lambda x: x.stem)[:-1]
santiago_files = sorted(santiago_files, key=lambda x: x.stem)[:-1]

dict_gens = {
    "S": santiago_files,
    "H": hugo_files,
    "Y": yosra_files,
    "C": clement_files,

}
annotators = read_mask_from_path_gens(dict_gens, REVERSE_BLIND_MAPPING)
annotators[modelname] = output

In [37]:
# fig, axes = imshow_compare(annotators, draw_bbox=True, max_images=8, ax_size=4)


In [38]:
skeletons = skeletonize_data_dict(annotators)
properties = extract_properties_from_datadict(skeletons)
df = pd.DataFrame.from_dict(properties)
df['common'] = df['S'].astype(int) + df['H'].astype(int) + df['Y'].astype(int) + df['C'].astype(int)  + df[modelname].astype(int) 

df.to_pickle("properties_with_IA.pkl")



In [39]:
df

,annotator,image_type,red,green,ratio,fiber_type,S,H,Y,C,SegFormer MiT-B4,common
0,S,siMMS22L_+dox-01,86,129,6.666667e-01,double,True,True,False,False,False,2
1,S,siMMS22L_+dox-01,226,211,1.071090e+00,multiple,True,False,False,False,False,1
2,S,siMMS22L_+dox-01,110,125,8.800000e-01,double,True,True,True,True,True,5
3,S,siMMS22L_+dox-01,21,38,5.526316e-01,double,True,False,True,True,True,4
4,S,siMMS22L_+dox-01,17,0,1.700000e+11,single,True,True,True,True,True,5
...,...,...,...,...,...,...,...,...,...,...,...,...
642,SegFormer MiT-B4,59_siBRCA2-02,11,18,6.111111e-01,double,False,False,False,False,True,1
643,SegFormer MiT-B4,59_siBRCA2-02,22,13,1.692308e+00,double,True,False,True,True,True,4
644,SegFormer MiT-B4,59_siBRCA2-02,40,55,7.272727e-01,double,False,False,False,False,True,1
645,SegFormer MiT-B4,59_siBRCA2-02,9,27,3.333333e-01,double,False,False,False,False,True,1
